In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/2022_AN2DL(Private)/ChallengeDL1/

#Import libraries

In [ ]:
# version of tensorflow needed to be able to use the ConvNeXtLarge
!pip install tensorflow==2.10.0
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

#Set seed for reproducibility

In [ ]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

#Suppress warnings

In [ ]:
import warnings
import logging

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)
tf.get_logger().setLevel('INFO')
tf.autograph.set_verbosity(0)

tf.get_logger().setLevel(logging.ERROR)
tf.get_logger().setLevel('ERROR')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

#Data loader

In [ ]:
# Load the dataset to be used for classification
!unzip training_dataset_homework1.zip

In [ ]:
#Install and run splitfolder
#Split folders: 85% of images in train folder, 15% in val folder
!pip install split-folders
import splitfolders
splitfolders.ratio('training_data_final', output='Dataset', seed=seed, ratio=(0.85,0.15))

In [ ]:
# Dataset folders
dataset_dir = 'training_data_final'
training_dir = 'Dataset/train'
validation_dir = 'Dataset/val'

# Images are divided into folders, one for each class.
# If the images are organized in such a way, we can exploit the
# ImageDataGenerator to read them from disk.
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an instance of ImageDataGenerator with Data Augmentation for the training data
aug_train_data_gen = ImageDataGenerator(# after many attempts, the most suitable values ​​identified are the following:
                                        rotation_range=30,
                                        height_shift_range=50,
                                        width_shift_range=50,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        fill_mode='reflect'
                                        ) # rescale value is not necessary with ConvNeXtLarge, normalization is included as
                                        # part of the model

val_gen = ImageDataGenerator(#rescale value is not necessary with ConvNeXtLarge,
    # normalization is included as part of the model
                             )

# Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method
#generation of training data
train_gen = aug_train_data_gen.flow_from_directory(directory=training_dir,
                                                       target_size=(96,96),
                                                       color_mode='rgb',
                                                       classes=None,
                                                       class_mode='categorical',
                                                       batch_size=25,
                                                       shuffle=True,
                                                       seed=seed)

#generation of validation data
valid_gen = val_gen.flow_from_directory(directory=validation_dir,
                                               target_size=(96,96),
                                               color_mode='rgb',
                                               classes=None,
                                               class_mode='categorical',
                                               batch_size=25,
                                               shuffle=False,
                                               seed=seed)

# Calculate class weights
from collections import Counter
counter = Counter(train_gen.classes)
max_val = float(max(counter.values()))
class_weights = {class_id: max_val/num_images for class_id, num_images in counter.items()}
print(class_weights)

#Models metadata

In [ ]:
#the size of each image in the dataset (96x96)
input_shape = (96, 96, 3)

epochs = 200
patience = 10

#CNN model with transfer learning

In [ ]:
# Apply same preprocessing used to train the supernet
supernet = tfk.applications.ConvNeXtLarge(
    include_top=False,
    weights="imagenet",
    input_shape=(224,224,3), #This model takes input images of shape (224, 224, 3)
    pooling="avg", #global average pooling will be applied to the output of the last convolutional layer.
    include_preprocessing=True,
)

supernet.trainable = False #first training only to our basic model

# Download and plot the model
supernet.summary()
tfk.utils.plot_model(supernet)

In [ ]:
def build_model(input_shape):

    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='input_layer')

    #resizing of each image from (96,96,3) to (224,224,3)
    x = tfkl.Resizing(224, 224, interpolation="bicubic", name='resizing')(input_layer)

    x = supernet(x)

    #use of a classifier layer composed of 4096 units
    classifier_layer = tfkl.Dense(units=4096, name='Classifier', kernel_initializer=tfk
                                  .initializers.HeUniform(seed), activation='relu')(x)

    #output layer classifies the 8 species
    output_layer = tfkl.Dense(units=8, activation='softmax', kernel_initializer=tfk.initializers
                              .GlorotUniform(seed), name='output_layer')(classifier_layer)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

    # Return the model
    return model

In [ ]:
# Build model
model = build_model(input_shape)
model.summary()

In [ ]:
# Train the model
model.fit
history = model.fit(
    x = train_gen,
    epochs = epochs,
    validation_data = valid_gen,
    class_weight = class_weights, #use of weights identified above
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=patience,
    restore_best_weights=True)]
).history

model.save("Models/ConvNew3")

In [ ]:
# Plot the training
plt.figure(figsize=(20,5))
plt.plot(history['loss'], label='Training', alpha=.8, color='#ff7f0e')
plt.plot(history['val_loss'], label='Validation', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Binary Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(20,5))
plt.plot(history['accuracy'], label='Training', alpha=.8, color='#ff7f0e')
plt.plot(history['val_accuracy'], label='Validation', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

#Fine Tuning

In [ ]:
# Re-load the model after transfer learning
ft_model = tfk.models.load_model('Models/ConvNew3')
netname = 'convnext_large'

In [ ]:
# Set all layers to True
ft_model.get_layer(netname).trainable = True

# Freeze first 80 layers
for i, layer in enumerate(ft_model.get_layer(netname).layers[:80]):
  layer.trainable=False

ft_model.summary()

# Compile the model with learning rate of 0.0001
ft_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-4), metrics='accuracy')

In [ ]:
# Train the model (fine tuning)
ft_model.fit
history = ft_model.fit(
    x = train_gen,
    epochs = epochs,
    validation_data = valid_gen,
    class_weight = class_weights,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=patience,
    restore_best_weights=True)]
).history

ft_model.save('Models/FinalModel')

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(history['loss'], label='Training', alpha=.8, color='#ff7f0e')
plt.plot(history['val_loss'], label='Validation', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Binary Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(20,5))
plt.plot(history['accuracy'], label='Training', alpha=.8, color='#ff7f0e')
plt.plot(history['val_accuracy'], label='Validation', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()